# Exercice 11

### Q1
Analysons le code de la fonction `perdecomp`

In [ ]:
import numpy as np
import imtools as im
import matplotlib.pyplot as plt

In [ ]:
# Image de niveaux de gris aléatoires, de taille 500 x 400
u = np.random.randint(low=0, high=256, size=(500, 400), dtype=int)

In [ ]:
# On récupère la taille de l'image dans les 2 dimensions
# ny, nx = 500, 400 ici
ny, nx = u.shape
u = u.astype('double')  # On convertit l'image entière en flottants
X = np.arange(nx)  # X = [0, 1, ..., nx - 1]
Y = np.arange(ny)  # Y = [0, 1, ..., ny - 1]
v = np.zeros((ny, nx))  # v est une image nulle de la même taille que u

# On calcule la différence des valeurs de u(x,y)
# avec les voisins extérieurs de u(x,y)

# La ligne supérieure de v est la différence entre
# la ligne supérieure de u - ligne inférieure de u
v[0, X] = u[0, X] - u[-1, X]
v[-1,X] = -v[0,X]  # ligne inférieure de v = opposé de sa ligne supérieure

# On effectue les mêmes opérations sur les colonnes gauche et droite
# de v, en prenant soin de ne pas écraser les valeurs aux coins
v[Y, 0] = v[Y, 0] + u[Y, 0] - u[Y, -1]
v[Y,-1] = v[Y,-1] - u[Y,0] + u[Y,-1]

# fx est la matrice M_ij = (cos(2pi i/nx))
# pour i = 0, ..., nx-1 et j = 0, ..., ny-1
# => toutes les lignes de fx sont identiques
fx = np.tile(np.cos(2.*np.pi*X/nx), (ny,1))

# fy est la matrice M_ij = (cos(2pi j/ny))
# pour i = 0, ..., nx-1 et j = 0, ..., ny-1
# => toutes les colonnes de j sont identiques
fy = np.tile(np.cos(2.*np.pi*Y/ny), (nx,1)).T
fx[0,0] = 0 # avoid division by 0 in the line below

Pour expliquer les lignes suivantes, considérons les objets entrant en jeu :

* l'image $v$ est une image nulle, sauf sur les bords, sur laquelle elle prend les valeurs de $u$, moins les valeurs des voisins extérieurs de ces points de $u$. Ceci correspond au laplacien extérieur de $u$, défini comme
\begin{equation*}
\begin{split}
v(x, y) = \Delta_{ext}(u)(x, y) &= 0 \text{ si } x \notin \{0, N_x - 1\} \text{ et } y \notin \{0, N_y - 1\}\\
&= u(x, y) - u(N_x - 1 - x, y) \text{ si } x \in \{0, N_x - 1\} \text{ et } y \notin \{0, N_y - 1\}\\
&= u(x, y) - u(x, N_y - 1 - y) \text{ si } y \in \{0, N_y - 1\} \text{ et } x \notin \{0, N_x - 1\}\\
&= 2u(x, y) - u(N_x - 1 - x, y) - u(x, N_y - 1 - y) \text{ si } y \in \{0, N_y - 1\} \text{ et } x \in \{0, N_x - 1\}
\end{split}
\end{equation*}
* D'après le théorème de décomposition periodic-smooth, cette image $v$ est égale au produit de convolution d'un noyau $\varphi$ avec la partie "smooth" de l'image $u$, notée $s$, ce qui donne en prenant la transformée de Fourier :
$$\Delta_{ext}(u) = \varphi * s \implies \hat{\Delta_{ext}(u)} = \hat{\varphi} \cdot \hat{s}$$
* Or, on a $\hat{\varphi}(x, y) = 4 - 2\text{cos}\left(\frac{2\pi x}{N_x}\right) - 2\text{cos}\left(\frac{2\pi y}{N_y}\right)$
* Par conséquent, la ligne `s = np.real(np.fft.ifft2(np.fft.fft2(v)*0.5/(2.-fx-fy)))` :
* * Calcule $\hat{v}$ la transformée de Fourier de `v` : `np.fft.fft2(v)`
* * Divise $\hat{v}$ par $\frac{1}{2} \frac{1}{2 - \left(\text{cos}\left(\frac{2\pi x}{N_x}\right) + \text{cos}\left(\frac{2\pi y}{N_y}\right)\right)}$, i.e. on calcule $\hat{s} = \frac{\hat{\Delta_{ext}(u)}}{\hat{\varphi}}$
* * On prend ensuite la partie réelle de la transformée de Fourier inverse de $\hat{v}$ pour obtenir la partie "smooth" de l'image $u$, et on extrait ainsi les composantes périodiques et smooth de $u$ dans les images `p` et `s`

In [ ]:
s = np.real(np.fft.ifft2(np.fft.fft2(v)*0.5/(2.-fx-fy)))
p = u-s

In [ ]:
def show_psdecomp(img, p=None, s=None):
    if p is None:
        p, s = im.perdecomp(img)

    _, ax = plt.subplots(1, 3, figsize=(12, 4))
    ax[0].imshow(img, cmap='gray')
    ax[0].set_title('Image de départ')
    ax[1].imshow(p, cmap='gray')
    ax[1].set_title('Partie périodique')
    ax[2].imshow(s, cmap='gray')
    ax[2].set_title('Partie smooth')

In [ ]:
show_psdecomp(u, p, s)

### Q2

In [ ]:
# Check u == p+s
print(f'u and p+s are strictly equal : {np.array_equal(u, p+s)}')
print(f'Max difference between u and p+s : {np.max(np.abs(u - (p+s)))}')

On peut conclure que sur cet exemple, $u$ et $p + s$ sont strictement égales à une erreur de précision numérique près, bien que théoriquement, nous nous attendions à une différence de l'ordre de $10^{-16}$ en précision double. Essayons avec des images plus régulières qu'un simple bruit

In [ ]:
lena = im.load('lena.pgm').astype('double')
lena_p, lena_s = im.perdecomp(lena)

In [ ]:
show_psdecomp(lena, lena_p, lena_s)

In [ ]:
print(f'lena and lena_p + lena_s are strictly equal : {np.array_equal(lena, lena_p + lena_s)}')
print(f'Max difference between lena and lena_p + lena_s : {np.max(np.abs(lena - (lena_p + lena_s)))}')

Il semble y avoir une erreur systématique mais très faible dans cette décomposition, puisqu'on obtient exactement la même différence entre les deux images. Essayons de localiser cette différence

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].imshow(np.abs(lena - (lena_p + lena_s)), cmap='gray')
ax[0].set_title('abs(lena - (lena_p + lena_s))')
ax[1].imshow(np.abs(u - (p + s)), cmap='gray')
ax[1].set_title('abs(u - (p + s))')

In [ ]:
im.View(np.hstack((lena, lena_p)))

Il est difficile de voir sur la figure ci-dessus les pixels non-nuls, mais on voit que leur localisation n'est pas identique pour les deux images et qu'ils dépendent des structures respectives de chaque image. L'erreur commise est donc certainement due à une propagation d'erreur de précision numérique, ou d'une approximation faite par l'algorithme FFT.

In [ ]:
_, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(np.tile(lena, (2, 2)), cmap='gray')
ax[0].set_title('Image originale')
ax[1].imshow(np.tile(lena_p, (2, 2)), cmap='gray')
ax[1].set_title('Partie périodique')
ax[2].imshow(np.tile(lena_s, (2, 2)), cmap='gray')
ax[2].set_title('Partie smooth')

La partie périodique de l'image n'a plus de discontinuités aux bords de l'image lorsque celle-ci est périodisée car on a isolé ces discontinuités dans la partie smooth, ce qui permet de résoudre les artefacts liés aux discontinuités dans le domaine de Fourier. On peut visualiser ces artefacts dans le domaine de Fourier

In [ ]:
lena_fft = np.fft.fftshift(np.fft.fft2(lena))
lena_p_fft = np.fft.fftshift(np.fft.fft2(lena_p))
lena_s_fft = np.fft.fftshift(np.fft.fft2(lena_s))

In [ ]:
_, ax = plt.subplots(2, 3, figsize=(12, 8))
ax[0][0].imshow(np.log(1 + np.abs(lena_fft)), cmap='gray')
ax[1][0].imshow(np.angle(lena_fft), cmap='gray')
ax[0][0].set_title('Image originale')

ax[0][1].imshow(np.log(1 + np.abs(lena_p_fft)), cmap='gray')
ax[1][1].imshow(np.angle(lena_p_fft), cmap='gray')
ax[0][1].set_title('Partie périodique')

ax[0][2].imshow(np.log(1 + np.abs(lena_s_fft)), cmap='gray')
ax[1][2].imshow(np.angle(lena_s_fft), cmap='gray')
ax[0][2].set_title('Partie smooth')

La partie smooth de l'image a donc absorbé tous les artefacts, en isolant les discontinuités à ses bords. Ainsi, on peut calculer une transformée de Fourier correcte après avoir périodisé l'image. On peut comparer ces résultats avec l'image symétrisée

In [ ]:
lena_sym = im.fsym2(lena)

In [ ]:
plt.imshow(lena_sym, cmap='gray')

In [ ]:
lena_sym_fft = np.fft.fftshift(np.fft.fft2(lena_sym))
_, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(np.log(1 + np.abs(lena_sym_fft)), cmap='gray')
ax[1].imshow(np.angle(lena_sym_fft), cmap='gray')

Bien que la symétrisation élimine des artefacts sur le module de la transformée de Fourier, ce n'est pas le cas de la phase. De plus, dans le cas d'une onde pure, impossible de retrouver l'onde originelle à partir de la transformée de Fourier de la symétrisée

In [ ]:
onde_fft = np.zeros(shape=(512, 512), dtype=float)
onde_fft[20, 20] = 1000
onde = np.real(np.fft.ifft2(onde_fft))
onde_sym = im.fsym2(onde)
onde_sym_fft = np.fft.fft2(onde_sym)

In [ ]:
_, ax = plt.subplots(1, 4, figsize=(16, 4))
ax[0].imshow(onde, cmap='gray')
ax[0].set_title('Originale')
ax[1].imshow(onde_sym, cmap='gray')
ax[1].set_title('Symétrisée')
ax[2].imshow(np.log(1 + np.abs(onde_sym_fft)), cmap='gray')
ax[2].set_title('mod(F(Symétrisée))')
ax[3].imshow(np.angle(onde_sym_fft), cmap='gray')
ax[3].set_title('phase(F(Symétrisée))')

Comme on peut le voir sur le module de la transformée de Fourier de la symétrisée de l'onde pure, il est impossible d'identifier le vecteur d'onde correct parmi les 4 pics d'intensité.

## Exercice 12

### Q1, Q2
Les propriétés $(i)$ et $(ii)$ découlent de la définition de $s$ et de l'égalité $\Delta s = \Delta_{\text{ext}} u$.

### Q3

Dans la preuve, on a montré :

$$s = \left(Q_1 + Q_2\right)^{-1} Q_1 \, u$$

où $Q_1 \in S^n_{+}$ et $Q_2 \in S^n_{++}$.

On a :

\begin{equation*}
\begin{split}
    p &= u - s\\
      &= u - \left(Q_1 + Q_2\right)^{-1} Q_1 \, u\\
      &= (I - \left(Q_1 + Q_2\right)^{-1} Q_1) \, u\\
      &= (\left(Q_1 + Q_2\right)^{-1} \left(Q_1 + Q_2\right) - \left(Q_1 + Q_2\right)^{-1} Q_1) \, u\\
      &= \left(Q_1 + Q_2\right)^{-1} Q_2 \, u
\end{split}
\end{equation*}

Or, $\left(Q_1 + Q_2\right)^{-1} Q_2$ est inversible, $u \mapsto \text{per}(u)$ est donc une bijection linéaire.

### Q4

Soit $\lambda$ une valeur propre de $\text{per}$, et $u_\lambda$ un vecteur propre associé (non nul).

On a :

\begin{equation*}
\begin{split}
    \text{per}(u_\lambda) = \lambda u_\lambda &= \left(Q_1 + Q_2\right)^{-1} Q_2 \, u_\lambda\\
    \iff \left(Q_1 + Q_2\right) \lambda u_\lambda &= Q_2 u_\lambda\\
    \iff \lambda \left( u_\lambda^* \left(Q_1 + Q_2\right) u_\lambda \right) &= u_\lambda^* Q_2 u_\lambda\\
    \iff \lambda &= \frac{u_\lambda^* Q_2 u_\lambda}{u_\lambda^* \left(Q_1 + Q_2\right) u_\lambda}
\end{split}
\end{equation*}

Le numérateur est strictement positif puisque $Q_2 \in S^n_{++}$, et le dénominateur est supérieur au numérateur puisque $Q_1 \in S^n_+$, d'où $\lambda \in ]0, 1]$

### Q5

Considérons $u \in \mathcal{P}$. Alors, on a $\Delta_{\text{ext}} u = 0 = \Delta s$. Or :

\begin{equation*}
\begin{split}
    \Delta s = 0 & \iff \varphi \ast s = 0\\
                 & \iff \hat{\varphi} \cdot \hat{s} = 0\\
                 & \iff \hat{s} = 0\\
                 & \iff s = 0\\
                 & \iff u = \text{per}(u)
\end{split}
\end{equation*}

Les points fixes de $\text{per}$ sont donc exactement les éléments de $\mathcal{P}$.

### Q6

Il existe une matrice $P$ inversible telle que $P^T \left( Q_1 + Q_2 \right) P = I$ et $P^T Q_2 P = D$ avec $D$ diagonale

On note $M$ la matrice de $\text{per}$ dans la base canonique. On a alors :

\begin{equation*}
\begin{split}
    \left( Q_1 + Q_2 \right)^{-1} Q_2 &= A\\
    \iff Q_2 &= \left( Q_1 + Q_2 \right) A
\end{split}
\end{equation*}

Donc :
\begin{equation*}
\begin{split}
    P^{-1} A P &= P^T \left( Q_1 + Q_2 \right) P P^{-1} A P\\
    &= P^T \left( \left( Q_1 + Q_2 \right)A \right) P\\
    &= P^T Q_2 P\\
    &= D
\end{split}
\end{equation*}

$\text{per}$ est donc diagonalisable.


# Exercice 14
### Q1

In [ ]:
u = im.load('lena.pgm').astype('double')
x0 = (u.shape[1] + 1) / 2
y0 = (u.shape[0] + 1) / 2 # centre de l’image
theta = np.pi / 6
u1 = im.fftshear(u, -np.tan(theta / 2), y0, axis=1)
u2 = im.fftshear(u1, np.sin(theta), x0, axis=0)
u3 = im.fftshear(u2, -np.tan(theta / 2), y0, axis=1)

In [ ]:
_, ax = plt.subplots(1, 4, figsize=(16, 4))
ax[0].imshow(u, cmap='gray')
ax[1].imshow(u1, cmap='gray')
ax[2].imshow(u2, cmap='gray')
ax[3].imshow(u3, cmap='gray')

In [ ]:
d = np.linspace(-1,1,500)
X,Y = np.meshgrid(d, d)
d = X+Y
x0 = (d.shape[1] + 1) / 2
y0 = (d.shape[0] + 1) / 2 # centre de l’image
d1 = im.fftshear(d, -np.tan(theta / 2), y0, axis=1)
d2 = im.fftshear(d1, np.sin(theta), x0, axis=0)
d3 = im.fftshear(d2, -np.tan(theta / 2), y0, axis=1)

In [ ]:
_, ax = plt.subplots(1, 4, figsize=(16, 4))
ax[0].imshow(d, cmap='gray')
ax[1].imshow(d1, cmap='gray')
ax[2].imshow(d2, cmap='gray')
ax[3].imshow(d3, cmap='gray')

Le schéma ci-dessous illustre comment les artefacts observés sont obtenus par glissements successifs :

In [ ]:
from IPython.display import Image
Image("schema_shear.jpg")

In [ ]:
# Try now with an angle > pi/2
theta = 5 * np.pi / 6
d1 = im.fftshear(d, -np.tan(theta / 2), y0, axis=1)
d2 = im.fftshear(d1, np.sin(theta), x0, axis=0)
d3 = im.fftshear(d2, -np.tan(theta / 2), y0, axis=1)

In [ ]:
_, ax = plt.subplots(1, 4, figsize=(16, 4))
ax[0].imshow(d, cmap='gray')
ax[1].imshow(d1, cmap='gray')
ax[2].imshow(d2, cmap='gray')
ax[3].imshow(d3, cmap='gray')

On observe une périodisation importante de l'image, ce qui est logique pour un angle important, plus précisément pour
$$\theta \geq 2\bar{\theta} = 2\text{ arctan}\left(\frac{2X}{Y}\right)$$
où $X$ et $Y$ sont la largeur et la hauteur de l'image : $\bar{\theta}$ représente l'angle où un glissement horizontal $G_{-\text{tan}(\bar{\theta})}$ produit une translation d'exactement une ligne aux extrémités de l'image, comme illustré ci-dessous.

In [ ]:
theta_bar = 2 * np.arctan(2 * len(d.T) / len(d))
theta = theta_bar

d1 = im.fftshear(d, -np.tan(theta / 2), y0, axis=1)
d2 = im.fftshear(d1, np.sin(theta), x0, axis=0)
d3 = im.fftshear(d2, -np.tan(theta / 2), y0, axis=1)

_, ax = plt.subplots(1, 4, figsize=(16, 4))
ax[0].imshow(d, cmap='gray')
ax[1].imshow(d1, cmap='gray')
ax[2].imshow(d2, cmap='gray')
ax[3].imshow(d3, cmap='gray')

La périodisation de l'image produit donc une image plus petite après rotation par succession de glissements. Il est en fait inutile de créer une rotation par succession de glissements d'une angle supérieur à un angle droit : il suffit d'effectuer la rotation complémentaire à l'angle droit, puis de tourner l'image discrète obtenue d'un angle droit (ou dans l'ordre inverse). Ainsi, on évite toute périodisation, et on obtient une image plus grande après rotation.

In [ ]:
d = lena
theta = 5 * np.pi / 6
d1 = im.fftshear(d, -np.tan(theta / 2), y0, axis=1)
d2 = im.fftshear(d1, np.sin(theta), x0, axis=0)
d3 = im.fftshear(d2, -np.tan(theta / 2), y0, axis=1)

_, ax = plt.subplots(2, 5, figsize=(20, 8))
ax[0][0].imshow(d, cmap='gray')
ax[0][1].imshow(d1, cmap='gray')
ax[0][2].imshow(d2, cmap='gray')
ax[0][3].imshow(d3, cmap='gray')
ax[0][3].set_title('Après glissements')
ax[0][4].imshow(d3, cmap='gray')
ax[0][4].set_title('Image finale (succession de glissements)')

# Now, use theta - np.pi / 2 as shearing angle,
# and rotate the image by a right angle
theta_2 = 5 * np.pi / 6 - np.pi / 2
d1 = im.fftshear(d, -np.tan(theta_2 / 2), y0, axis=1)
d2 = im.fftshear(d1, np.sin(theta_2), x0, axis=0)
d3 = im.fftshear(d2, -np.tan(theta_2 / 2), y0, axis=1)
d4 = np.rot90(d3, axes=(1, 0))

ax[1][0].imshow(d, cmap='gray')
ax[1][1].imshow(d1, cmap='gray')
ax[1][2].imshow(d2, cmap='gray')
ax[1][3].imshow(d3, cmap='gray')
ax[1][3].set_title('Après glissements')
ax[1][4].imshow(d4, cmap='gray')
ax[1][4].set_title('Image finale (glissements + rotation 90°)')

L'algorithme final de rotation d'une image d'un angle arbitraire $\theta \in [-\pi, \pi]$ est donc :

1. Poser
\begin{equation}
\begin{split}
\tilde{\theta} & = \theta \text{ si } \theta \in \left[ -\frac{\pi}{2}, \frac{\pi}{2} \right]\\
               & = \theta - \frac{\pi}{2} \text{ si } \theta > \frac{\pi}{2}\\
               & = \theta + \frac{\pi}{2} \text{ si } \theta < -\frac{\pi}{2}\\
\end{split}
\end{equation}

2. Effectuer une rotation d'angle $\theta - \tilde{\theta}$ sur l'image (i.e. rotation d'un angle droit si $\theta$ dépasse l'angle droit)
3. Effectuer une rotation d'angle $\tilde{\theta}$ par succession de glissements

In [ ]:
d = X+Y
x0 = (d.shape[1] + 1) / 2
y0 = (d.shape[0] + 1) / 2 # centre de l’image

In [ ]:
theta = np.pi / 12
_, ax = plt.subplots(6, 4, figsize=(16, 24))

for j in range(1, 7):
    d1 = im.fftshear(d, -np.tan(j * theta / 2), y0, axis=1)
    d2 = im.fftshear(d1, np.sin(j * theta), x0, axis=0)
    d3 = im.fftshear(d2, -np.tan(j * theta / 2), y0, axis=1)
    _ax = ax[j - 1]
    _ax[0].imshow(d, cmap='gray')
    _ax[1].imshow(d1, cmap='gray')
    _ax[2].imshow(d2, cmap='gray')
    _ax[3].imshow(d3, cmap='gray')

Le domaine d'une image carrée sur lequel toute rotation est exacte semble être confiné au plus grand carré inscrit dans le domaine de l'image, orienté d'une angle de $\frac{\pi}{4}$ par rapport au domaine de l'image